In [1]:
import json
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


file_path = "/kaggle/input/health/healthcare.json"
with open(file_path) as file:
    data = json.load(file)

# Extract patterns and tags
patterns = []
tags = []
responses = {}

for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])
    responses[intent['tag']] = intent['responses']

# Encode the tags
label_encoder = LabelEncoder()
encoded_tags = label_encoder.fit_transform(tags)

# Tokenize the patterns
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(patterns)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(patterns)
padded_sequences = pad_sequences(sequences, padding="post")

# Prepare input and output data
input_data = np.array(padded_sequences)
output_data = tf.keras.utils.to_categorical(encoded_tags)

# Build the model
model = Sequential([
    Embedding(input_dim=1000, output_dim=16, input_length=input_data.shape[1]),
    LSTM(32, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dense(32, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Train the model
history = model.fit(input_data, output_data, epochs=170, batch_size=8)

# Save the model and tokenizer
model.save("/kaggle/working/mentalhealth_chatbot_model_final.h5.keras")
import pickle
with open("/kaggle/working/tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle)
with open("/kaggle/working/label_encoder.pickle", "wb") as enc:
    pickle.dump(label_encoder, enc)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.0096 - loss: 4.3764  
Epoch 2/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0463 - loss: 4.1576 
Epoch 3/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0352 - loss: 3.9761 
Epoch 4/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0306 - loss: 3.8756 
Epoch 5/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0974 - loss: 3.6769
Epoch 6/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0799 - loss: 3.5426 
Epoch 7/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0585 - loss: 3.3322
Epoch 8/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0956 - loss: 3.2491
Epoch 9/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1240 - loss: 3.1241 
Epoch 10/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1068 - loss: 3.0817
Epoch 11/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1833 - loss: 2.9154
Epoch 12/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - ac

In [3]:
import random

# Load the trained model and tokenizer
from tensorflow.keras.models import load_model

model = load_model("/kaggle/working/mentalhealth_chatbot_model_final.h5.keras")
with open("/kaggle/working/tokenizer.pickle", "rb") as handle:
    tokenizer = pickle.load(handle)
with open("/kaggle/working/label_encoder.pickle", "rb") as enc:
    label_encoder = pickle.load(enc)

# Chat function
def chatbot_response(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=input_data.shape[1], padding="post")
    prediction = model.predict(padded)
    tag = label_encoder.inverse_transform([np.argmax(prediction)])
    return random.choice(responses[tag[0]])

# Test the chatbot
print("Chatbot is ready! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    response = chatbot_response(user_input)
    print("Bot:", response)


Chatbot is ready! Type 'quit' to exit.


You:  hello


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
Bot: Hi there. How are you feeling today?


You:  im sick right now


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: Good afternoon. How is your day going?


You:  not good


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Bot: Come Come elucidate your thoughts


You:  i have stomache


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: I'm here for you. Could you tell me why you're feeling this way?


You:  quit
